In [2]:
import requests 
from bs4 import BeautifulSoup
from urllib.request import urlopen 


In [17]:
res = requests.get("https://www.youtube.com/@krishnaik06/videos")
page_src = BeautifulSoup(res.text, 'html.parser')
page_src

<!DOCTYPE html>
<html darker-dark-theme="" darker-dark-theme-deprecate="" lang="en-GB" style="font-size: 10px;font-family: Roboto, Arial, sans-serif;" system-icons="" typography="" typography-spacing=""><head><script data-id="_gd" nonce="yVobsqzRmHOJiyUBWBPrjQ">window.WIZ_global_data = {"MuJWjd":false,"nQyAE":{},"oxN3nb":{"1":false}};</script><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="ApvK67ociHgr2egd6c2ZjrfPuRs8BHcvSggogIOPQNH7GJ3cVlyJ1NOq/COCdj0+zxskqHt9HgLLETc8qqD+vwsAAABteyJvcmlnaW4iOiJodHRwczovL3lvdXR1YmUuY29tOjQ0MyIsImZlYXR1cmUiOiJQcml2YWN5U2FuZGJveEFkc0FQSXMiLCJleHBpcnkiOjE2OTUxNjc5OTksImlzU3ViZG9tYWluIjp0cnVlfQ==" http-equiv="origin-trial"/><script nonce="yVobsqzRmHOJiyUBWBPrjQ">var ytcfg={d:function(){return window.yt&&yt.config_||ytcfg.data_||(ytcfg.data_={})},get:function(k,o){return k in ytcfg.d()?ytcfg.d()[k]:o},set:function(){var a=arguments;if(a.length>1)ytcfg.d()[a[0]]=a[1];else{var k;for(k in a[0])ytcfg.d()[k]=a[0][k]}}};
window.ytcfg.set('EME

In [27]:
div_content = page_src.find_all("div")


In [32]:
div_content

[<div class="ytd-searchbox-spt" id="search-container" slot="search-container"></div>,
 <div class="ytd-searchbox-spt" id="search-input" slot="search-input"><input autocapitalize="none" autocomplete="off" autocorrect="off" hidden="" id="search" name="search_query" spellcheck="false" tabindex="0" type="text"/></div>,
 <div id="masthead-logo" slot="masthead-logo"><a href="/" style="display: none;" title="YouTube"><svg class="external-icon" id="yt-logo-updated-svg" viewbox="0 0 90 20"><g id="yt-logo-updated" preserveaspectratio="xMidYMid meet" viewbox="0 0 90 20"><g><path d="M27.9727 3.12324C27.6435 1.89323 26.6768 0.926623 25.4468 0.597366C23.2197 2.24288e-07 14.285 0 14.285 0C14.285 0 5.35042 2.24288e-07 3.12323 0.597366C1.89323 0.926623 0.926623 1.89323 0.597366 3.12324C2.24288e-07 5.35042 0 10 0 10C0 10 2.24288e-07 14.6496 0.597366 16.8768C0.926623 18.1068 1.89323 19.0734 3.12323 19.4026C5.35042 20 14.285 20 14.285 20C14.285 20 23.2197 20 25.4468 19.4026C26.6768 19.0734 27.6435 18.1068

In [40]:
div_content[16]

<div class="video-details"><div class="thumbnail skeleton-bg-color"></div><div class="details flex-1"><div class="video-title text-shell skeleton-bg-color"></div><div class="video-meta text-shell skeleton-bg-color"></div></div></div>